In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
# Helper funtion to show an image with cv2
def cv_show_img(title, image, wait=0):
    cv2.namedWindow(title)
    cv2.startWindowThread()
    cv2.imshow(title, image)
    cv2.waitKey(wait)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# Helper funtion to show multiple images at the same time
def cv_show_mult_img(titleArr, imageArr, wait=0):
    for i in range(len(titleArr)):
        cv2.namedWindow(titleArr[i])
        cv2.startWindowThread()
        cv2.imshow(titleArr[i], imageArr[i])
    cv2.waitKey(wait)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

## Countours
Find bounded sets of pixels with a substantial difference in value to neighboring pixels.

Returned as sets of points that define the outer edge of the identified difference in pixels

In [3]:
img = cv2.imread('./images/bunchofshapes.jpg')
cv_show_img('Shapes Image', img)

In [6]:
# Pre-Processing to find contours

# 1. grayscale the image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 2. Find Canny Edges
canny = cv2.Canny(gray, 30, 200)

# 3. Find Countours - remember to use a copy of the image because find contours will alter the image
canny_copy = canny.copy()
contours, hierarchy = cv2.findContours(canny_copy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print(f'Contours Found:\t{len(contours)}')

# 4. Draw the contours on the original image
img_copy = img.copy()
cv2.drawContours(
    img_copy, 
    contours, 
    -1,         # -1 means draw all contours regardless of hierarchy
    (0,255,0),  # Color for the lines (green)
    thickness=5
)

cv_show_mult_img(
    ['Original','1. Grayscale','2. Canny Edges','3. Contour Identification','4. Draw Contours'],
    [img,gray,canny,canny_copy,img_copy]
)

Contours Found:	4


## Sorting & Approximating Contours
Sorting contours refers to identifying contours on an image

Involves finding the centroid of contours and sorting them by their coordinates & area

Approximating contours refers to approximating polygons for efficieincy reasons

#### Sorting By Area

In [7]:
# Sort contours by area
def get_contour_areas(contours):
    areas = []
    for c in contours:
        areas.append(cv2.contourArea(c))
    return areas

print(f'Areas Before Sorting:\t{get_contour_areas(contours)}')

# Sort contours
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
print(f'Areas After Sorting:\t{get_contour_areas(sorted_contours)}')

# Draw contours one at a time
img_copy = img.copy()
for c in range(len(sorted_contours)):
    cv2.drawContours(img_copy, [sorted_contours[c]], -1, (0,255,0), 5)
    cv_show_img(f'Contours {c}', img_copy)

Areas Before Sorting:	[20587.5, 22901.5, 66579.5, 90222.0]
Areas After Sorting:	[90222.0, 66579.5, 22901.5, 20587.5]


#### Sorting Left to right

In [8]:
def x_coord_contour(contours):
    '''Return X Coordinate for the centroid'''
    if cv2.contourArea(contours) > 10:
        moment = cv2.moments(contours)
        return (int(moment['m10']/moment['m00']))
    else:
        pass

def label_contour_center(image, c):
    '''Place a red circle at the centroid'''
    moment = cv2.moments(c)
    cx = int(moment['m10']/moment['m00'])
    cy = int(moment['m01']/moment['m00'])
    cv2.circle(image,(cx,cy),10,(0,0,255),-1)
    return image

img_copy = img.copy()
cv_show_img('Original',img_copy)

# get the object centers and draw the circles
for (i, c) in enumerate(contours):
    centers = label_contour_center(img_copy, c)
cv_show_img('Labeling Centers',centers)

# Sort Left to Right
contours_left_to_right = sorted(contours, key=x_coord_contour, reverse=False)

# Labeling Left to Right
for (i, c) in enumerate(contours_left_to_right):
    cv2.drawContours(img_copy, [c], -1, (0,255,0), 5)
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.putText(img_copy, f'# {i+1}', (cx,cy), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3)
    cv_show_img(f'Sorting Left to Right Step {i+1}',img_copy)

## Convex Hull
Correct for convexity defects

In [9]:
# Use the image of a hand to demo
hand = cv2.imread('./images/hand.jpg')
gray = cv2.cvtColor(hand, cv2.COLOR_BGR2GRAY)

#threshold
ret, thresh = cv2.threshold(gray, 176, 255, 0)

# contours
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# Sort contours by area, remove largest one
n = len(contours) - 1
contours = sorted(contours, key=cv2.contourArea, reverse=False)[:n]

# Iterate over contours and draw convex hull
hand_copy = hand.copy()
for c in contours:
    hull = cv2.convexHull(c)
    cv2.drawContours(hand_copy, [hull], 0, (0,255,0), 3)
cv_show_mult_img(
    ['Original','Grayscale','Thresholds','Convex Hull'],
    [hand, gray, thresh, hand_copy]
)

: 